In [1]:
import pandas as pd
import numpy
import os
import datetime

print(os.chdir('./'))


df = pd.read_csv('../data/Walmart_Store_sales.csv',low_memory=False)
print(df.head())
print(df.shape)
print(df.dtypes)

None
   Store        Date  Weekly_Sales  Holiday_Flag  Temperature  Fuel_Price  \
0    6.0  18-02-2011    1572117.54           NaN        59.61       3.045   
1   13.0  25-03-2011    1807545.43           0.0        42.38       3.435   
2   17.0  27-07-2012           NaN           0.0          NaN         NaN   
3   11.0         NaN    1244390.03           0.0        84.57         NaN   
4    6.0  28-05-2010    1644470.66           0.0        78.89       2.759   

          CPI  Unemployment  
0  214.777523         6.858  
1  128.616064         7.470  
2  130.719581         5.936  
3  214.556497         7.346  
4  212.412888         7.092  
(150, 8)
Store           float64
Date             object
Weekly_Sales    float64
Holiday_Flag    float64
Temperature     float64
Fuel_Price      float64
CPI             float64
Unemployment    float64
dtype: object


150 lignes pour 8 colonnes, petit volume de données 

In [9]:
print("Description des données : ")
print(df.columns)
print(df.describe())

Description des données : 
Index(['Store', 'Date', 'Weekly_Sales', 'Holiday_Flag', 'Temperature',
       'Fuel_Price', 'CPI', 'Unemployment'],
      dtype='object')
            Store  Weekly_Sales  Holiday_Flag  Temperature  Fuel_Price  \
count  150.000000  1.360000e+02    138.000000   132.000000  136.000000   
mean     9.866667  1.249536e+06      0.079710    61.398106    3.320853   
std      6.231191  6.474630e+05      0.271831    18.378901    0.478149   
min      1.000000  2.689290e+05      0.000000    18.790000    2.514000   
25%      4.000000  6.050757e+05      0.000000    45.587500    2.852250   
50%      9.000000  1.261424e+06      0.000000    62.985000    3.451000   
75%     15.750000  1.806386e+06      0.000000    76.345000    3.706250   
max     20.000000  2.771397e+06      1.000000    91.650000    4.193000   

              CPI  Unemployment  
count  138.000000    135.000000  
mean   179.898509      7.598430  
std     40.274956      1.577173  
min    126.111903      5.143000 

# Exploration des données

In [3]:
print("On explore les données...")

On explore les données...


# Data Cleaning

In [4]:
print("Affichage des du nombre de valeurs manquantes par colonnes :")
print(df.isna().sum())

Affichage des du nombre de valeurs manquantes par colonnes :
Store            0
Date            18
Weekly_Sales    14
Holiday_Flag    12
Temperature     18
Fuel_Price      14
CPI             12
Unemployment    15
dtype: int64


In [20]:
print("On drop les Weekly_Sales à zéro qui est la valeur à déduire pour éviter les biais")
new_df = df.copy()
print(new_df.shape)
new_df = new_df.fillna(new_df.mean(numeric_only=True))
print(new_df.isna().sum())

print(new_df.shape)

new_df = new_df.dropna()

print(new_df.isna().sum())
print(new_df.shape)


On drop les Weekly_Sales à zéro qui est la valeur à déduire pour éviter les biais
(150, 8)
Store            0
Date            18
Weekly_Sales     0
Holiday_Flag     0
Temperature      0
Fuel_Price       0
CPI              0
Unemployment     0
dtype: int64
(150, 8)
Store           0
Date            0
Weekly_Sales    0
Holiday_Flag    0
Temperature     0
Fuel_Price      0
CPI             0
Unemployment    0
dtype: int64
(132, 8)


## Il manque toujours 18 dates
Je décide de les supprimer 

In [6]:
new_df = new_df.dropna(how='any')
print(f"La nouvelle shape de données est : {new_df.shape}")

La nouvelle shape de données est : (132, 8)


In [7]:
print("Conversion des dates")
#Changement simple des infos car bien formaté à la base
new_df['Date'] = pd.to_datetime(new_df['Date'])
print(new_df['Date'].dtype)
print(new_df['Date'])
new_df['year'] = new_df['Date'].dt.year
new_df['month'] = new_df['Date'].dt.month
new_df['day'] = new_df['Date'].dt.day
new_df['day_of_week'] = new_df['Date'].dt.dayofweek
new_df.head()

Conversion des dates
datetime64[ns]
0     2011-02-18
1     2011-03-25
2     2012-07-27
4     2010-05-28
5     2010-05-28
         ...    
144   2012-10-19
145   2010-06-18
147   2010-06-11
148   2011-08-12
149   2012-04-20
Name: Date, Length: 132, dtype: datetime64[ns]


C:\Users\bonna\AppData\Local\Temp\ipykernel_32936\52456130.py:3: UserWarning: Parsing dates in %d-%m-%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  new_df['Date'] = pd.to_datetime(new_df['Date'])


,Store,Date,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment,year,month,day,day_of_week
0,6.0,2011-02-18,1.572118e+06,0.07971,59.610000,3.045000,214.777523,6.858,2011,2,18,4
1,13.0,2011-03-25,1.807545e+06,0.00000,42.380000,3.435000,128.616064,7.470,2011,3,25,4
2,17.0,2012-07-27,1.249536e+06,0.00000,61.398106,3.320853,130.719581,5.936,2012,7,27,4
4,6.0,2010-05-28,1.644471e+06,0.00000,78.890000,2.759000,212.412888,7.092,2010,5,28,4
5,4.0,2010-05-28,1.857534e+06,0.00000,61.398106,2.756000,126.160226,7.896,2010,5,28,4


In [8]:
print("drop des outliers")
print(f"Shape du dataFrame : {new_df.shape}")
outliers_col = ['Temperature', 'Fuel_Price', 'CPI', 'Unemployment']
for col in outliers_col:
    mean = new_df[col].mean()
    ecart_type = new_df[col].std()
    bas, haut = mean - 3*ecart_type, mean + 3*ecart_type
    new_df = new_df[(new_df[col]>= bas) & (new_df[col] <= haut)]
print(f"Shape du dataFrame après supp des outliers : {new_df.shape}")


drop des outliers
Shape du dataFrame : (132, 12)
Shape du dataFrame après supp des outliers : (127, 12)
